In [1]:
import pandas as pd

pre_separated_data = True  # Setar para False quando os dados de cadastro não estiverem separados em dataframes menores

## EDA

**Dados de treino (2022)**

Este é o conjunto de dados que você e sua equipe irão usar para trabalhar, criar o modelo, fazer testes e desenvolver a solução final. 

Transações: Data, PDV, Produto, Quantidade, Faturamento. 

Cadastro de produtos: Produto, Categoria, Descrição, + até 4 atributos.

Cadastro de PDVs: PDV, On/Off Prem, Categoria (c-store, g-store, liquor etc.), Zipcode. 

In [2]:
transacoes_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet"
cadastro_prod_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-6364321654468257203-dc13a5d6-36ae-48c6-a018-37d8cfe34cf6-263-1-c000.snappy.parquet"
cadastro_pdv_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet"

Separando o .parquet original do cadastro de produtos, para posteriormente acessá-lo via pandas:

In [3]:
if not pre_separated_data:
    import polars as pl

    df3 = pl.read_parquet(cadastro_prod_path)
    print(df3.shape)

    total_rows = df3.shape[0]
    chunk_size = total_rows // 10  # Integer division to get the chunk size

    for i in range(10):
        start_index = i * chunk_size
        end_index = start_index + chunk_size

        if i == 9:
            chunk = df3.slice(start_index)
        else:
            chunk = df3.slice(start_index, chunk_size)

        output_filename = (
            cadastro_prod_path.strip(".snappy.parquet") + f"_part_{i + 1}.parquet"
        )

        chunk.write_parquet(output_filename)
        print(f"Written {output_filename}")

In [50]:
df_transacoes = pd.read_parquet(transacoes_path)
df_cadastro_pdv = pd.read_parquet(cadastro_pdv_path)

In [ ]:
if pre_separated_data:
    all_dfs = []
    for i in range(10):
        df3 = pd.read_parquet(
            cadastro_prod_path.strip(".snappy.parquet") + f"_part_{i + 1}.parquet"
        )
        all_dfs.append(df3)
    df_cadastro_prod = pd.concat(all_dfs, ignore_index=True)
    del all_dfs

In [ ]:
df_transacoes.shape, df_cadastro_pdv.shape, df_cadastro_prod.shape

((6560698, 11), (14419, 4), (192356316, 8))

### **Identificando se há linhas duplicadas nos dataframes:**

In [ ]:
df_cadastro_pdv.drop_duplicates(inplace=True)
df_cadastro_prod.drop_duplicates(inplace=True)
df_transacoes.drop_duplicates(inplace=True)

df_cadastro_pdv.shape, df_cadastro_prod.shape, df_transacoes.shape

((14419, 4), (304956, 8), (6560698, 12))

### **Identificando se há valores nulos / faltantes nos dataframes**

In [34]:
df_cadastro_pdv.isnull().sum()

pdv              0
premise          0
categoria_pdv    0
zipcode          0
dtype: int64

In [ ]:
df_cadastro_prod.isnull().sum()  # Aparentemente a coluna label não invalida o uso desses dados, já é que ela especifica uma característica categória do produto (que pode ou não ser usada).

produto             0
categoria           0
descricao           0
tipos               0
label           63339
subcategoria        0
marca               0
fabricante          0
dtype: int64

In [32]:
df_cadastro_prod

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,69753381296344216,Package,MONDAY NIGHT BREW DON'T CALL IT HOTLANTA 6/4/1...,Package,In&Out,Red Wine,Monday Night Don't Call It Hotlanta,Monday Night Brewing
1,5019314249828979377,Distilled Spirits,BARRELL V907 RED PHONE BOOTH & TOWER,Distilled Spirits,Specialty,Lager / Pilsner,Barrell Bourbon- Private Label,Barrell Craft Spirits
2,4016404282141162328,Distilled Spirits,DRINKWORKS SIMPLY REFRESHING GIN & TONIC 12/4/...,Distilled Spirits,Core,Lager,Drinkworks Simply Refreshing Gin & Tonic,AB Drinkworks
3,6217366559810422145,Distilled Spirits,CENZON REPOSADO 100% AGAVE 6/750ML 80PF,Distilled Spirits,None,Lager,Cenzon Reposado 100% Agave Tequila,Sazerac Spirits
4,7356488787409434558,Draft,REFORMATION STARK 1/4 KEG,Draft,Core,Red Wine,Reformation Stark,Reformation Brewery
...,...,...,...,...,...,...,...,...
183621172,6964240149904972585,Distilled Spirits,PAUL MASSON PEACH BRANDY 6/1.75L 54PF,Distilled Spirits,None,Ready To Drink Cocktails,Paul Masson Peach Brandy,Sazerac Spirits
183621178,5759670436535233385,Distilled Spirits,MR BOSTON BLUE CURACAO 12/1L 30PF,Distilled Spirits,None,Ready To Drink Cocktails,Mr. Boston Blue Curacao Liqueur,Sazerac Spirits
183621205,8225211493147767417,Wine,SANS WINE CO. - POOR RANCH VINEYARDS ROSE OF C...,Wine < 14 %,Discontinued,Ready To Drink Cocktails,Poor Ranch Vineyards Rose Of Carignan,Sans Wine Company
183621218,3128727580621798365,Package,MICHELOB ULTRA PRICKLY PEAR LIME 4/6/12 LN,Package,Core,Ready To Drink Cocktails,Michelob Ultra Prickly Pear Lime,AB Anheuser Busch Inc


In [ ]:
df_cadastro_prod.groupby("produto").value_counts()

produto              categoria  descricao                                    tipos        label         subcategoria      marca                                fabricante         
1000423277513436210  Draft      REFORMATION A COLD ONE 97.1 PILSNER 1/4 KEG  Draft        Core          Ale               Reformation A Cold One 97.1 Pilsner  Reformation Brewery    1
                                                                                                        Bourbon Whiskey   Reformation A Cold One 97.1 Pilsner  Reformation Brewery    1
                                                                                                        Brandy / Cognac   Reformation A Cold One 97.1 Pilsner  Reformation Brewery    1
                                                                                                        Canadian Whisky   Reformation A Cold One 97.1 Pilsner  Reformation Brewery    1
                                                                                     

In [ ]:
df_cadastro_prod[df_cadastro_prod.produto == "998529431391212014"].head(
    10
)  # Para um mesmo produto, há variações na descrição da subcategoria

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
65742025,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Bourbon Whiskey,Lyrarakis Assyrtiko Voila,Lyrarakis
65746450,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Other Spirits,Lyrarakis Assyrtiko Voila,Lyrarakis
65747335,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Other Wine,Lyrarakis Assyrtiko Voila,Lyrarakis
65749105,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Tequila / Mezcal,Lyrarakis Assyrtiko Voila,Lyrarakis
65751760,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Vodka,Lyrarakis Assyrtiko Voila,Lyrarakis
65755300,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Liqueurs & Cordials,Lyrarakis Assyrtiko Voila,Lyrarakis
65756185,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Ready-to-Drink,Lyrarakis Assyrtiko Voila,Lyrarakis
65772115,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Scotch Whisky,Lyrarakis Assyrtiko Voila,Lyrarakis
65777425,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Canadian Whisky,Lyrarakis Assyrtiko Voila,Lyrarakis
65811055,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Brandy / Cognac,Lyrarakis Assyrtiko Voila,Lyrarakis


In [ ]:
df_cadastro_prod.groupby("produto").size()

produto
1000423277513436210    43
1000794189319388082    43
1000817480846862155    43
1004943868572044494    43
1005164764874054592    43
                       ..
996135836802743128     43
997545005038320324     43
997691518031628045     43
998529431391212014     43
999285078291803499     43
Length: 7092, dtype: int64

In [ ]:
# Agrupando o dataframe pelo produto, checar quantas variações de subcategoria e categoria existem para cada produto
df_cadastro_prod.groupby("produto").agg(
    {
        "subcategoria": "nunique",
        "categoria": "nunique",
        "fabricante": "nunique",
        "label": "nunique",
    }
).reset_index().rename(
    columns={
        "subcategoria": "num_subcategorias",
        "categoria": "num_categorias",
        "fabricante": "num_fabricantes",
        "label": "num_labels",
    }
).sort_values(by=["num_subcategorias", "num_categorias"], ascending=False)

,produto,num_subcategorias,num_categorias,num_fabricantes,num_labels
0,1000423277513436210,43,1,1,1
1,1000794189319388082,43,1,1,1
2,1000817480846862155,43,1,1,1
3,1004943868572044494,43,1,1,1
4,1005164764874054592,43,1,1,1
...,...,...,...,...,...
7087,996135836802743128,43,1,1,1
7088,997545005038320324,43,1,1,1
7089,997691518031628045,43,1,1,0
7090,998529431391212014,43,1,1,1


In [31]:
df_transacoes.isnull().sum()

internal_store_id      0
internal_product_id    0
distributor_id         0
transaction_date       0
reference_date         0
quantity               0
gross_value            0
net_value              0
gross_profit           0
discount               0
taxes                  0
semana                 0
dtype: int64

In [6]:
df_transacoes.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758


`internal_store_id` de `df_transacoes` corresponde ao `pdv` da tabela `df_cadastro_pdv`. \
`internal_product_id` de `df_transacoes` corresponde ao `produto` da tabela `df_cadastro_prod`.

In [7]:
df_cadastro_pdv.head()

,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093


In [8]:
df_cadastro_prod.head()

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,69753381296344216,Package,MONDAY NIGHT BREW DON'T CALL IT HOTLANTA 6/4/1...,Package,In&Out,Red Wine,Monday Night Don't Call It Hotlanta,Monday Night Brewing
1,5019314249828979377,Distilled Spirits,BARRELL V907 RED PHONE BOOTH & TOWER,Distilled Spirits,Specialty,Lager / Pilsner,Barrell Bourbon- Private Label,Barrell Craft Spirits
2,4016404282141162328,Distilled Spirits,DRINKWORKS SIMPLY REFRESHING GIN & TONIC 12/4/...,Distilled Spirits,Core,Lager,Drinkworks Simply Refreshing Gin & Tonic,AB Drinkworks
3,6217366559810422145,Distilled Spirits,CENZON REPOSADO 100% AGAVE 6/750ML 80PF,Distilled Spirits,None,Lager,Cenzon Reposado 100% Agave Tequila,Sazerac Spirits
4,7356488787409434558,Draft,REFORMATION STARK 1/4 KEG,Draft,Core,Red Wine,Reformation Stark,Reformation Brewery


Arquivo de previsão no formato CSV ou Parquet, com as seguintes colunas:

| semana | pdv  | produto | quantidade |
|--------|------|---------|------------|
| 1      | 1023 | 123     | 120        |
| 2      | 1045 | 234     | 85         |
| 3      | 1023 | 456     | 110       |

No caso do csv, utilize “;” como caractere separador (exemplo: 1;1023;123;120) e encoding UTF-8.

**semana (número inteiro)**: número da semana (1 a 4 de janeiro/2023) \
**pdv (número inteiro)**: código do ponto de venda \
**produto (número inteiro)**: código do SKU \
**quantidade (número inteiro)**: previsão de vendas


## Organizar dados para obter Y

In [ ]:
df_transacoes["mes"] = pd.to_datetime(df_transacoes["transaction_date"]).dt.month
df_transacoes["ano"] = pd.to_datetime(df_transacoes["transaction_date"]).dt.year

# Criar a coluna que define a semana em um dado mês do ano: semana 1 = dias 1-7, semana 2 = dias 8-14, semana 3 = dias 15-21, semana 4 = dias 22-28, semana 5 = dias 29-31
df_transacoes["semana"] = (
    (pd.to_datetime(df_transacoes["transaction_date"]).dt.day - 1) // 7
) + 1
#df_transacoes["semana"] = (
#    (pd.to_datetime(df_transacoes["transaction_date"]).dt.dayofyear - 1) // 7
#) + 1

In [78]:
df_transacoes.semana.value_counts()

semana
2    1765689
3    1471955
1    1418108
4    1408951
5     495995
Name: count, dtype: int64

In [79]:
df_transacoes.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes,semana,mes,ano,dia
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375,2,7,2022,13
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000,3,3,2022,21
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000,1,9,2022,6
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000,2,9,2022,11
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758,3,2,2022,18


In [80]:
df_transacoes.value_counts(["semana", "transaction_date"]).sort_index().head(30)

semana  transaction_date
1       2022-01-01            170
        2022-01-02           1112
        2022-01-03           8833
        2022-01-04          20934
        2022-01-05          18039
        2022-01-06          17456
        2022-01-07          12957
        2022-02-01          22134
        2022-02-02          19963
        2022-02-03          28452
        2022-02-04          11448
        2022-02-05            606
        2022-02-06           1786
        2022-02-07          24453
        2022-03-01          23408
        2022-03-02          22547
        2022-03-03          19214
        2022-03-04          16378
        2022-03-05            202
        2022-03-06           3027
        2022-03-07          26414
        2022-04-01          17229
        2022-04-02            328
        2022-04-03           1754
        2022-04-04          30538
        2022-04-05          26130
        2022-04-06          24412
        2022-04-07          19417
        2022-05-01     

In [101]:
# Ao agrupar, manter as colunas mes e ano
df_transacoes_grouped = (
    df_transacoes.groupby(["semana","ano", "mes", "internal_store_id", "internal_product_id"]) # Excluí o distributor_id do agrupamento, assumindo que não será necessário em nossa análise
    .agg({"quantity": "sum",
    "gross_value": "sum",
    "net_value": "sum",
    "gross_profit": "sum",
    "discount": "sum",
    "taxes": "sum",
})
    .reset_index()
)
df_transacoes_grouped = df_transacoes_grouped.rename(
    columns={
        "internal_store_id": "pdv",
        "internal_product_id": "produto",
        "quantity": "quantidade",
    }
)

# Find rows that contain equal pdv and produto with same semana
(df_transacoes.groupby(["semana","mes", "ano", "internal_store_id", "internal_product_id"]).semana.value_counts() > 1).sum() # Cerca 278k casos em que, para um dado PDV+produto, foi vendido mais de uma vez na mesma semana

np.int64(278549)

In [102]:
df_transacoes_grouped.sort_values(by=["ano", "mes"])

,semana,ano,mes,pdv,produto,quantidade,gross_value,net_value,gross_profit,discount,taxes
0,1,2022,1,1001371918471115422,1029370090212151375,2.0,54.299999,49.740486,16.224486,0.000000,4.559516
1,1,2022,1,1001371918471115422,1120490062981954254,18.0,262.350006,234.992905,62.210903,53.099998,27.357101
2,1,2022,1,1001371918471115422,2239307647969388381,2.0,37.950001,31.870647,8.182648,5.100000,6.079354
3,1,2022,1,1001371918471115422,4353552881410365573,7.0,138.250000,118.302116,33.567116,2.800000,19.947886
4,1,2022,1,1001371918471115422,4797439216678436447,1.0,26.750000,24.470242,7.524241,0.000000,2.279758
...,...,...,...,...,...,...,...,...,...,...,...
6275222,5,2022,12,996297376394222790,9024979450937837276,1.0,24.600000,24.600000,8.560000,0.000000,0.000000
6275223,5,2022,12,997907800111849739,3516587220187345473,9.0,220.725006,218.615631,51.665630,24.750000,2.109375
6275224,5,2022,12,997907800111849739,4451855961536069898,5.0,164.250000,163.078125,46.283127,0.000000,1.171875
6275225,5,2022,12,997907800111849739,7029448080469668619,5.0,164.250000,163.078125,46.283127,0.000000,1.171875


In [103]:
df_transacoes

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes,semana,mes,ano,dia
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375,2,7,2022,13
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000,3,3,2022,21
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000,1,9,2022,6
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000,2,9,2022,11
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758,3,2,2022,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560693,1439579822593789678,2523145330235258825,4,2022-12-06,2022-12-01,3.0,63.000000,61.379761,20.879761,0.000000,1.620240,1,12,2022,6
6560694,2357589266242432199,3148628429396585683,4,2022-05-20,2022-05-01,1.0,12.300000,12.300000,3.650000,2.800000,0.000000,3,5,2022,20
6560695,7245187752757348055,6668439675743950745,4,2022-08-17,2022-08-01,1.0,33.849998,31.570240,10.191240,0.000000,2.279758,3,8,2022,17
6560696,8952609383990245450,3894706280449257667,5,2022-02-03,2022-02-01,4.0,74.556000,73.655998,17.991999,15.600000,0.900000,1,2,2022,3


In [104]:
df_cadastro_prod.produto.unique().shape

(7092,)

In [105]:
df_cadastro_prod.drop_duplicates().shape

(7092, 7)

## Salvando o dataframe de cadastro filtrado pelo id do produto

In [ ]:
df_cadastro_prod = df_cadastro_prod.drop_duplicates(subset=["produto"])
#df_cadastro_prod.to_parquet(
#    "/data/challenges/hackathon_bigdata_2025/processed_data/cadastro_produtos.parquet",
#    index=False,
#)

## Lendo o novo dataframe de cadastro de produtos pré-processado e unindo-o com os outros:

In [106]:
df_cadastro_prod = pd.read_parquet("/data/challenges/hackathon_bigdata_2025/processed_data/cadastro_produtos.parquet")

In [107]:
df_cadastro_pdv

,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093
...,...,...,...,...
14414,5047149578012031122,Off Premise,Package/Liquor,81501
14415,8352177618955815208,Off Premise,Package/Liquor,80219
14416,7754182004766664314,On Premise,Special Event,81004
14417,9044929558186581877,On Premise,Other On Premise,81226


In [108]:
rows_before_merge = df_transacoes_grouped.shape[0]
df_transacoes_grouped = df_transacoes_grouped.merge(
    df_cadastro_pdv, left_on="pdv", right_on="pdv", how="inner",
)
rows_after_merge = df_transacoes_grouped.shape[0]
rows_before_merge, rows_after_merge, rows_before_merge - rows_after_merge # Cerca de 39k linhas são perdidas no merge ao usar "inner", indicando que há certos PDV_ids não presentes no dataframe de cadastro de PDVs.

(6275227, 6235791, 39436)

In [109]:
df_transacoes_grouped

,semana,ano,mes,pdv,produto,quantidade,gross_value,net_value,gross_profit,discount,taxes,premise,categoria_pdv,zipcode
0,1,2022,1,1001371918471115422,1029370090212151375,2.0,54.299999,49.740486,16.224486,0.000000,4.559516,Off Premise,Convenience,30175
1,1,2022,1,1001371918471115422,1120490062981954254,18.0,262.350006,234.992905,62.210903,53.099998,27.357101,Off Premise,Convenience,30175
2,1,2022,1,1001371918471115422,2239307647969388381,2.0,37.950001,31.870647,8.182648,5.100000,6.079354,Off Premise,Convenience,30175
3,1,2022,1,1001371918471115422,4353552881410365573,7.0,138.250000,118.302116,33.567116,2.800000,19.947886,Off Premise,Convenience,30175
4,1,2022,1,1001371918471115422,4797439216678436447,1.0,26.750000,24.470242,7.524241,0.000000,2.279758,Off Premise,Convenience,30175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6235786,5,2022,12,996297376394222790,9024979450937837276,1.0,24.600000,24.600000,8.560000,0.000000,0.000000,Off Premise,Convenience,30035
6235787,5,2022,12,997907800111849739,3516587220187345473,9.0,220.725006,218.615631,51.665630,24.750000,2.109375,Off Premise,Package/Liquor,80205
6235788,5,2022,12,997907800111849739,4451855961536069898,5.0,164.250000,163.078125,46.283127,0.000000,1.171875,Off Premise,Package/Liquor,80205
6235789,5,2022,12,997907800111849739,7029448080469668619,5.0,164.250000,163.078125,46.283127,0.000000,1.171875,Off Premise,Package/Liquor,80205


In [110]:
full_df = df_transacoes_grouped.merge(
    df_cadastro_prod, on="produto", how="inner"
)

In [111]:
full_df.drop(columns=["internal_store_id", "internal_product_id"], errors="ignore", inplace=True)
full_df.head()

,semana,ano,mes,pdv,produto,quantidade,gross_value,net_value,gross_profit,discount,taxes,premise,categoria_pdv,zipcode,categoria,descricao,tipos,label,marca,fabricante
0,1,2022,1,1001371918471115422,1029370090212151375,2.0,54.299999,49.740486,16.224486,0.000000,4.559516,Off Premise,Convenience,30175,Package,MICHELOB ULTRA 2/12/12 CN,Package,Core,Michelob Ultra,AB Anheuser Busch Inc
1,1,2022,1,1001371918471115422,1120490062981954254,18.0,262.350006,234.992905,62.210903,53.099998,27.357101,Off Premise,Convenience,30175,Package,BUD LIGHT 2/12/8 CN,Package,Discontinued,Bud Light,AB Anheuser Busch Inc
2,1,2022,1,1001371918471115422,2239307647969388381,2.0,37.950001,31.870647,8.182648,5.100000,6.079354,Off Premise,Convenience,30175,Package,NATURAL LIGHT 4/6/16 CN,Package,Core,Natural Light,AB Anheuser Busch Inc
3,1,2022,1,1001371918471115422,4353552881410365573,7.0,138.250000,118.302116,33.567116,2.800000,19.947886,Off Premise,Convenience,30175,Package,NATURAL LIGHT 2/15/12 CN,Package,Core,Natural Light,AB Anheuser Busch Inc
4,1,2022,1,1001371918471115422,4797439216678436447,1.0,26.750000,24.470242,7.524241,0.000000,2.279758,Off Premise,Convenience,30175,Package,BUD LIGHT LIME 2/12/12 CN,Package,Core,Bud Light Lime,AB Anheuser Busch Inc


In [114]:
cols = ['pdv', 'zipcode', 'produto', 'semana', 'mes', 'ano', 'categoria_pdv', 'categoria', 'premise', 'tipos', 'label', 'marca', 'fabricante', 'descricao',
       'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes', 'quantidade']
full_df = full_df[cols]
full_df.head()

,pdv,zipcode,produto,semana,mes,ano,categoria_pdv,categoria,premise,tipos,label,marca,fabricante,descricao,gross_value,net_value,gross_profit,discount,taxes,quantidade
0,1001371918471115422,30175,1029370090212151375,1,1,2022,Convenience,Package,Off Premise,Package,Core,Michelob Ultra,AB Anheuser Busch Inc,MICHELOB ULTRA 2/12/12 CN,54.299999,49.740486,16.224486,0.000000,4.559516,2.0
1,1001371918471115422,30175,1120490062981954254,1,1,2022,Convenience,Package,Off Premise,Package,Discontinued,Bud Light,AB Anheuser Busch Inc,BUD LIGHT 2/12/8 CN,262.350006,234.992905,62.210903,53.099998,27.357101,18.0
2,1001371918471115422,30175,2239307647969388381,1,1,2022,Convenience,Package,Off Premise,Package,Core,Natural Light,AB Anheuser Busch Inc,NATURAL LIGHT 4/6/16 CN,37.950001,31.870647,8.182648,5.100000,6.079354,2.0
3,1001371918471115422,30175,4353552881410365573,1,1,2022,Convenience,Package,Off Premise,Package,Core,Natural Light,AB Anheuser Busch Inc,NATURAL LIGHT 2/15/12 CN,138.250000,118.302116,33.567116,2.800000,19.947886,7.0
4,1001371918471115422,30175,4797439216678436447,1,1,2022,Convenience,Package,Off Premise,Package,Core,Bud Light Lime,AB Anheuser Busch Inc,BUD LIGHT LIME 2/12/12 CN,26.750000,24.470242,7.524241,0.000000,2.279758,1.0


In [115]:
full_df.isna().sum()

pdv                   0
zipcode               0
produto               0
semana                0
mes                   0
ano                   0
categoria_pdv         0
categoria             0
premise               0
tipos                 0
label            513272
marca                 0
fabricante            0
descricao             0
gross_value           0
net_value             0
gross_profit          0
discount              0
taxes                 0
quantidade            0
dtype: int64

Temos agora um dataframe que engloba todos os metadados disponibilizados, com algumas ressalvas:

1. Há valores nulos na coluna label;
2. Foram retirados cerca de 39k internal_store_id/PDVs que não estavam englobados no dataframe de Cadastro de PDVs para cruzar os metadados. Como esse cadastro fornece informações importantes, consideramos excluir esses PDVs sem cadastro. 
3. Foi excluída a coluna distributor_id, assumindo-se que não será uma informação relevante (após se agrupar os dados por mes,semana,ano, e IDs).

In [116]:
full_df.to_parquet("/data/challenges/hackathon_bigdata_2025/processed_data/transacoes_merged.parquet")

In [117]:
full_df.shape

(6235791, 20)